# Crawling Data Twitter dan Web

## Twitter

### Snscrape

In [41]:
_= !pip install snscrape

In [42]:
import snscrape.modules.twitter as sntwitter
import pandas as pd

In [43]:
def tweet_scrape(tag, count=100):
  list_tweet = []
  for i,tweet in enumerate(sntwitter.TwitterHashtagScraper(tag).get_items()):
    if i > count:
      break
    list_tweet.append({
      'Nama': tweet.user.username, 
      'Content': tweet.content, 
      'Lokasi': tweet.user.location
    })
  return pd.DataFrame(list_tweet) 

In [44]:
tweet = tweet_scrape('capres2024 AND ganjarpranowo')
tweet.to_csv('ganjarScrape.csv', index=False)

<ipython-input-43-cbc57a5e4014>:8: FutureWarning: content is deprecated, use rawContent instead
  'Content': tweet.content,


### Twint

In [45]:
_= !pip3 uninstall twint && pip3 install twint && pip3 install --user --upgrade git+https://github.com/twintproject/twint.git@origin/master#egg=twint

In [46]:
_= !pip install nest_asyncio

In [47]:
import twint
import nest_asyncio
nest_asyncio.apply()

In [48]:
c = twint.Config()
c.Search = '#capres2024 AND #ganjarpranowo'
c.Lang = 'id'
c.Limit = 1000
# c.Since = "2019-01-01"
# c.Until = "2022-01-01"
c.Pandas = True
twint.run.Search(c)

1627593902260727810 2023-02-20 09:00:00 +0000 <GanjaranApp> Refleksi sejarah Bung Karno menjadi spirit perjuangan untuk masa depan Indonesia. Maju terus @ganjarpranowo sebagai Presiden Indonesia 2024 demi masa depan Indonesia Maju ✊  #ganjarpranowo #ganjarsiap #ganjaranapp #ganjarkita #capres2024 #RelawanGanjarPranowo  https://t.co/FP9fQdy6nW
1627503307253248001 2023-02-20 03:00:00 +0000 <GanjaranApp> "Pejabat yang gemar nonton teater sehat kejiwaannya. Karena menjadi kaidah juga dalam teater sebagai seni mengkritik penguasa" Butet Kertaredjasa  #seniman #seni #teater #butet #butetkartredjasa #ganjarpranowo #ganjarsiap #ganjaranapp #ganjarkita #capres2024  https://t.co/MdW5FVyJUR
1627486484013150208 2023-02-20 01:53:09 +0000 <beritafaktaind> Ganjar Pranowo Cinta Budaya Tanah Air. #ganjarpranowo #budaya #budayaindonesia #pemilu2024 #capres2024 #beritafaktaind  Bu Mega, Indomie, Kakao, Minum, Iphone 14, Rusia  https://t.co/WhqfL1WHml
1627231520258760704 2023-02-19 09:00:01 +0000 <Ganjara

In [49]:
Tweets_df = twint.storage.panda.Tweets_df

In [50]:
Tweets_df.columns

Index(['id', 'conversation_id', 'created_at', 'date', 'timezone', 'place',
       'tweet', 'language', 'hashtags', 'cashtags', 'user_id', 'user_id_str',
       'username', 'name', 'day', 'hour', 'link', 'urls', 'photos', 'video',
       'thumbnail', 'retweet', 'nlikes', 'nreplies', 'nretweets', 'quote_url',
       'search', 'near', 'geo', 'source', 'user_rt_id', 'user_rt',
       'retweet_id', 'reply_to', 'retweet_date', 'translate', 'trans_src',
       'trans_dest'],
      dtype='object')

In [51]:
df = pd.DataFrame({
    'name': Tweets_df['name'],
    'tweet': Tweets_df['tweet'],
    'place': Tweets_df['place'],
})

In [52]:
df.to_csv('ganjarCapres.csv')

## Web

In [53]:
_= !pip install bs4

In [54]:
import requests

In [55]:
from bs4 import BeautifulSoup

In [56]:
import pandas as pd

### pta.trunojoyo.ac.id

In [57]:
class ScrapingPtaTrunojoyo:
  def __init__(self, url):
    self.url = url
  
  def page(self):
    res = requests.get(self.url)
    soup = BeautifulSoup(res.content, 'html.parser')
    contents = soup.find('ol', class_='pagination').findChildren(recursive=False)
    return int(contents[-3].text)

  def artikel_page(self, page):
    res = requests.get(f'{self.url}/{page}')
    soup = BeautifulSoup(res.content, 'html.parser')
    contents = soup.find_all('li', attrs={'data-cat':'#luxury'})
    return contents

  def get_berita(self):
    pta_list = []
    page = self.page()
    for pg in range(1, page+1):
      artikels = self.artikel_page(pg)
      for artikel in artikels:
        respone = requests.get(artikel.find('a', class_='gray button')['href'])
        soup_data = BeautifulSoup(respone.content, 'html.parser')
        cont = soup_data.find('div', attrs={'id':'content_journal'})
        # ambil data
        nama = cont.findAll('div')[1].div.span.text.split(' : ')[1]
        judul = str.join("", (cont.find('a', class_='title').string).splitlines()).capitalize()
        abstrak = str.join("", (cont.find('p').string).splitlines()).capitalize()
        pta_list.append({
            'Penulis' : nama,
            'Judul': judul,
            'Abstrak': abstrak
        })
    self.pta = pta_list
  
  def result(self):
    return pd.DataFrame(self.pta)
  
  def save(self, namefile='pta'):
    df = pd.DataFrame(self.pta).to_csv(namefile+'.csv', index=False)
    print('Berhasil di save')

In [58]:
scPta = ScrapingPtaTrunojoyo('https://pta.trunojoyo.ac.id/c_search/byprod/10/')

In [59]:
scPta.get_berita()

In [60]:
scPta.result()

,Penulis,Judul,Abstrak
0,A.Ubaidillah S.Kom,Perancangan dan implementasi sistem database t...,Sistem informasi akademik (siakad) merupaka...
1,"M. Basith Ardianto,",Aplikasi kontrol dan monitoring jaringan kompu...,Berjalannya koneksi jaringan komputer dengan l...
2,"Akhmad Suyandi, S.Kom",Rancang bangun aplikasi proxy server untukenkr...,Web server adalah sebuah perangkat lunak serve...
3,Heri Supriyanto,Sistem pendukung keputusan optimasi penjadwala...,Penjadwalan kuliah di perguruan tinggi me...
4,Septian Rahman Hakim,Sistem augmented reality animasi benda bergera...,Seiring perkembangan teknologi yang ada diduni...
5,Adi Chandra Laksono,Gerak pekerja pada game real time strategy men...,Gerak pekerja ada pada game yang memiliki genr...
6,NURRACHMAT,Rancang bangun game perawatan sapi karapan men...,"Perkembangan game yang semakin pesat, memberik..."
7,Muhammad Choirur Rozi,Ekstraksi fitur berbasis two dimensional linea...,Sistem pengenalan wajah adalah suatu sistem un...
8,M Khoiril Anwar,Implementasi algoritma prim dan depth first ...,Teknologi mobile game beroperating system open...
9,MALIKUL HAMZAH,Perancangan sistem informasi badan kepegawaian...,Kantor badan kepegawaian kota bangkalan adalah...


In [61]:
scPta.save()

Berhasil di save


### Kompas

In [62]:
class ScrapingKompas:
  def __init__(self, url):
    self.url = url

  def page(self):
    res_ = requests.get(self.url)
    soup_ = BeautifulSoup(res_.content, 'html.parser')
    contents_ = soup_.findAll('a', class_='paging__link')[:-2]
    return len(contents_)

  def artikel_page(self, page):
    res_ = requests.get(f'{self.url}?page={page}')
    soup_ = BeautifulSoup(res_.content, 'html.parser')
    contents_ = soup_.findAll('a', class_='article__link')
    return contents_

  def get_berita(self):
    berita_list = []
    page = self.page()
    for pg in range(1, page+1):
      artikels = self.artikel_page(pg)
      for artikel in artikels:
        res = requests.get(artikel['href'])
        soup = BeautifulSoup(res.content, 'html.parser')
        # ambil data
        judul = soup.find('h1', class_='read__title').string
        isi = isi = ' '.join(soup.find('div', class_='read__content').stripped_strings)
        berita_list.append({
            'Judul': judul,
            'Isi Berita': isi
        })
    self.beritas = berita_list
    return 
  
  def result(self):
    return pd.DataFrame(self.beritas)
  
  def save(self, namefile='beritaKompas'):
    df = pd.DataFrame(self.beritas).to_csv(namefile+'.csv', index=False)
    print('Berhasil di save')

In [63]:
data = ScrapingKompas('https://indeks.kompas.com')

In [64]:
data.get_berita()

In [65]:
data.result()

,Judul,Isi Berita
0,Sopir Fortuner yang Tertembak Senpi Majikan Te...,"JAKARTA, KOMPAS.com - Seorang sopir berinisial..."
1,15 Jurusan Baru UI di SNBP 2023 dan UTBK SNBT ...,KOMPAS.com - Universitas Indonesia ( UI ) memb...
2,4 Orang yang Tewas Akibat Ledakan di Blitar Te...,KOMPAS.com - Polisi bersama tim SAR berhasil m...
3,"Lirik Lagu You and Me, Singel Baru dari Vedo","JAKARTA, KOMPAS.com – Penyanyi dan penulis lag..."
4,16 Makanan yang Sebaiknya Dihindari Penderita ...,KOMPAS.com - Ginjal merupakan salah satu organ...
5,Komedian Lee Kyeong Shil Dilaporkan atas Dugaa...,"KOMPAS.com - Komedian senior di Korea Selatan,..."
6,Detik-detik Penumpang Helikopter Dievakuasi Le...,KOMPAS.com - Proses evakuasi para penumpang he...
7,Festival Musik Open Air Amphitheatre Pertama d...,KOMPAS.com – Dua penyanyi muda kenamaan Indone...
8,Komentar Pengunjung IIMS 2023 Soal Wuling Alvez,"JAKARTA, KOMPAS.com - Wuling Alvez resmi melun..."
9,"Sinopsis Iron Mask, Melancong ke China Demi Me...","JAKARTA, KOMPAS.com - Iron Mask merupakan film..."


In [66]:
data.save()

Berhasil di save


### Detik.com

In [67]:
class ScrapingDetik:
  def __init__(self, url):
    self.url = url
  
  def page(self):
    res_ = requests.get(self.url)
    soup_ = BeautifulSoup(res_.content, 'html.parser')
    contents_ = soup_.find('div', class_='pagination').findChildren()
    return int(contents_[-2].text)

  def artikel_page(self, page):
    res_ = requests.get(f'{self.url}/{page}')
    soup_ = BeautifulSoup(res_.content, 'html.parser')
    contents_ = soup_.findAll('a', class_='media__link')
    return contents_
  
  def get_berita(self):
    berita_list = []
    page = 5  # self.page()
    for pg in range(1, page+1):
      artikels = self.artikel_page(pg)
      for artikel in range(1, len(artikels), 2):
        res = requests.get(artikels[artikel]['href'])
        soup = BeautifulSoup(res.content, 'html.parser')
        # hapus tag style script
        for d in soup(['style', 'script']):
          d.decompose()
        # ambil data
        judul = ' '.join(soup.find('h1', class_='detail__title').stripped_strings)
        isi = ' '.join(soup.find('div', class_='detail__body-text').stripped_strings)
        berita_list.append({
            'Judul': judul,
            'Isi Berita': isi
        })
    self.beritas = berita_list

  def result(self):
    return pd.DataFrame(self.beritas)
  
  def save(self, namefile='beritaDetik'):
    df = pd.DataFrame(self.beritas).to_csv(namefile+'.csv', index=False)
    print('Berhasil di save')

In [68]:
scrap = ScrapingDetik('https://news.detik.com/indeks')

<a class="media__link" href="https://news.detik.com/pemilu/d-6575185/ketum-ppp-ngaku-siap-hadapi-pemilu-proporsional-terbuka-atau-tertutup" onclick='_pt(this, "newsfeed", "Ketum PPP Ngaku Siap Hadapi Pemilu Proporsional Terbuka atau Tertutup", "artikel 1")'>
<span class="ratiobox ratiobox--4-3 lqd">
<img alt="Ketum PPP Ngaku Siap Hadapi Pemilu Proporsional Terbuka atau Tertutup" class="" src="https://awsimages.detik.net.id/community/media/visual/2023/02/17/ketum-ppp-mardiono_43.jpeg?w=210&amp;q=90" title="Ketum PPP Ngaku Siap Hadapi Pemilu Proporsional Terbuka atau Tertutup"> </img></span>
</a>,

In [69]:
scrap.get_berita()

In [70]:
scrap.result()

,Judul,Isi Berita
0,Polisi: Pemilik Fortuner Tak Sengaja Tembak So...,Jakarta - Pria inisial AM yang merupakan sopir...
1,"Cari Solusi Tembok Rawan Jebol, Giliran Bu Ami...",Jakarta - Tembok Bu Ami di Tebet Jakarta Selat...
2,Airlangga Tunggu Arahan Jokowi soal Pengganti ...,Jakarta - Zainudin Amali disebut siap melepas ...
3,BSSN Ungkap Pencurian Data Jadi Serangan Siber...,Jakarta - Badan Siber dan Sandi Negara ( BSSN ...
4,"Cegah Keluarga Saling Gugat di Pengadilan, Lur...",Jakarta - Sengketa waris merupakan salah satu ...
...,...,...
95,30 Twibbon Hari Peduli Sampah Nasional 2023: L...,Jakarta - Twibbon Hari Peduli Sampah Nasional ...
96,Bareskrim Minta Polda Sulsel Usut Pengedar Nar...,Jakarta - Salah satu tersangka pengedar narkob...
97,Ini Alasan Polisi Kabulkan Penanguhan Penahana...,Jakarta - Polisi menangguhkan penahanan pengem...
98,Jusuf Kalla Pantau Langsung Proses Evakuasi He...,Wakil Presiden Republik Indonesia (Wapres RI) ...


In [71]:
scrap.save()

Berhasil di save
